In [5]:
from datasets import load_dataset
from transformers import (
    MarianTokenizer,
    MarianMTModel,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
import evaluate

In [6]:
# Load dataset and subset
print("Loading WMT14 dataset...")
dataset = load_dataset("wmt14", "de-en")
raw_dataset = dataset["train"].select(range(100))
split = raw_dataset.train_test_split(test_size=0.1)
train_raw = split["train"]
val_raw = split["test"]

# Load model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-de"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

Loading WMT14 dataset...


In [13]:

# Preprocess with batching
def preprocess_batch(batch):
    # Extract source (EN) and target (DE) sentences
    src_texts = [ex["en"] for ex in batch["translation"]]
    tgt_texts = [ex["de"] for ex in batch["translation"]]

    # Tokenize source
    model_inputs = tokenizer(src_texts, max_length=128, truncation=True, padding="max_length")

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(tgt_texts, max_length=128, truncation=True, padding="max_length")

    # Attach labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


print("Preprocessing data...")
train_dataset = train_raw.map(preprocess_batch, batched=True)
val_dataset = val_raw.map(preprocess_batch, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# BLEU scoring
bleu = evaluate.load("bleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Make sure predictions are decoded from flat sequences
    if isinstance(preds[0], list) and isinstance(preds[0][0], list):
        preds = [p[0] for p in preds]  # flatten nested prediction

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 with pad_token_id in labels before decoding
    labels = [[(token if token != -100 else tokenizer.pad_token_id) for token in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Format for BLEU
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["bleu"]}


# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./marian-finetuned-en-de",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    save_strategy="epoch",
    save_total_limit=1,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Preprocessing data...


/var/folders/qx/kwt2yzc57djflb63h7j_sxg40000gn/T/ipykernel_4707/1003572296.py:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [14]:
# Train and save
print("Training model...")
trainer.train()




Training model...


Epoch,Training Loss,Validation Loss


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [ ]:

# Save your fine-tuned model
model.save_pretrained("./marian-finetuned-en-de")
tokenizer.save_pretrained("./marian-finetuned-en-de")
print("\n Training complete and model saved to ./marian-finetuned-en-de")

('./marian-finetuned-en-de/tokenizer_config.json',
 './marian-finetuned-en-de/special_tokens_map.json',
 './marian-finetuned-en-de/vocab.json',
 './marian-finetuned-en-de/source.spm',
 './marian-finetuned-en-de/target.spm',
 './marian-finetuned-en-de/added_tokens.json')